## GMM Estimation of Logit Model



### Introduction



Consider a &ldquo;logit&rdquo; regression; score function from MLE is:
$$
     \frac{1}{N}\sum_j X_j\left(y_j - \frac{e^{X_j\beta}}{1+e^{X_j\beta}}\right) = 0.
  $$
Let $u_j = y_j - \frac{e^{X_j\beta}}{1+e^{X_j\beta}}$; if we have some $Z$ such that
$\mbox{E}(u|Z) = 0$ then we can construct further moment conditions
$$
     \frac{1}{N}\sum_j Z_j\left(y_j - \frac{e^{X_j\beta}}{1+e^{X_j\beta}}\right) = 0.
  $$



### GMM Estimator



In [5]:
import numpy as np

# Some different options for moments...

def mle(b,y,X,Z):
    """Observations of score for MLE estimator.

    Moment condition is E(y_j - p(x_jb))x_j = 0,
    where p(xb) = e^{xb}/(1+e^{xb})
    """
    p = np.exp(X*b)  # This is actually the odds
    p = p/(1+p)      # This is probability y=1

    return X*(y - p)

def nonlinear_iv(b,y,X,Z):
    """Observations for restriction that Z
    orthogonal to score.

    Moment condition is E(Z_jy_j - Z_jexp(x_jb)) = 0
    """
    p = np.exp(X*b)  # This is actually the odds
    p = p/(1+p)      # This is probability y=1

    return Z*(y - p)

### Data Generating Process



In [2]:
from scipy.stats import distributions as iid

def dgp(N,beta,VXZ,gamma=1):
    """Generate a tuple of (y,X,Z).

    Satisfies model:
        Pr(y=1|X) = f(X@beta,gamma)
        u = y - f(X@beta,gamma)
        E(u|Z) = 0
        f(x,gamma) = (exp(x)/(1+exp(x)))**gamma
        Var([X,Z}) = VXZ
        X,Z mean zero, Gaussian

    Each element of the tuple is an array of N observations.
    When gamma=1 this reduces to the logit model

    Inputs include
    - beta :: Governs effect of X on probability y=1
    - gamma :: Governs curvature of function
    - VXZ :: Var([X,Z])
    """
    
    # "Square root" of VXZ via eigendecomposition
    lbda,v = np.linalg.eig(VXZ)
    SXZ = v@np.diag(np.sqrt(lbda))

    # Generate normal random variates [X*,Z]
    XZ = iid.norm.rvs(size=(N,VXZ.shape[0]))@SXZ.T

    X = XZ[:,[0]] 
    Z = XZ[:,1:]

    # Calculate y
    pi = np.exp(X*beta)
    pi = (pi/(1+pi))**gamma

    y = iid.bernoulli(pi).rvs(size=(N,1))

    return y,X,Z

### The Truth (Mark I)



In this version of the truth the form of the distribution of $y$ is
known up to the parameter $\beta$; MLE takes advantage of this.

Choose some parameters to establish the &ldquo;truth&rdquo;:



In [3]:
import numpy as np
from numpy.linalg import inv

## Play with us!
beta = 2     # "Coefficient of interest"

## Play with us!

# Let Z have order ell, and X order 1, with Var([X,Z]|u)=VXZ

ell = 1 # Play with me too!

# Arbitrary (but deterministic) choice for VXZ
A = np.sqrt(1/np.arange(1,(ell+1)**2+1)).reshape((ell+1,ell+1)) 

## Below here we're less playful.

# Var([X,Z]|u) is constructed so that pos. def.
VXZ = A.T@A 

truth = (beta,VXZ)

### Monte Carlo Analysis of MLE via GMM



Here we take the score to be our moment condition; $Z$ doesn&rsquo;t appear,
so estimator is just identified.



In [10]:
import gmm # Just code defined last class

gmm.gj = mle  # Redefine gj function to use MLE score

data = dgp(10000,*truth)

soltn = gmm.one_step_gmm(data)

limiting_J = iid.chi2(0)

print("b=%f, J=%f, Critical J=%f" % (soltn[0],soltn[1],limiting_J.isf(0.05)))

b=0.282076, J=0.000000, Critical J=nan


Now our experiment begins.  We set our frequentist hats firmly on our
heads, and draw repeated samples of data, each generating a
corresponding estimate of beta.  Then the empirical distribution of
these samples tells us about the *finite* sample performance of our estimator.

We&rsquo;ll generate a sample of estimates of $b$ by drawing repeated
samples of size $N$, until estimates of the covariance of our
estimates converge:



In [ ]:
import gmm # Just code defined last class

N = 1000 # Sample size

tol = 1e-3

b_mle = []
b_nliv = []
J_nliv = []

d=0
oldV = 0
newV = 1
while d<30 or np.linalg.norm(oldV-newV) > tol:
    d += 1
    oldV = newV
    data = dgp(N,*truth)
    gmm.gj = mle
    soltn_mle = gmm.two_step_gmm(data)
    b_mle.append(soltn_mle.x)

    gmm.gj = nonlinear_iv
    soltn_nliv = gmm.two_step_gmm(data)
    b_nliv.append(soltn_nliv.x)
    J_nliv.append(soltn_nliv.fun)

    newV = np.var(b_nliv)

Now compare MLE & NLIV estimates:



In [ ]:
from matplotlib import pyplot as plt

_ = plt.scatter(b_mle,b_nliv)